In [10]:
import requests
import json
import pprint
import random

IP = "39.7.230.30"

In [11]:
# session/resource

resource = requests.get("http://20.196.214.79:5050/session/resource", params={"ip": IP})
print("resource :", resource.status_code)
print("resource :", resource.json()["message"])

resource : 200
resource : Opened


In [12]:
# session/create

params = {
  "ip": IP,
  "isWindowMode": True,
  "Res_X": 1920,
  "Res_Y": 1080
}
create = requests.post(f"http://20.196.214.79:5050/session/create", params)
print("createRequest :", create.content)
key = json.loads(create.content)['key']

In [7]:
# session/join and game/start

params = {"key": key, "playername": "dohyun"}
join = requests.post("http://20.196.214.79:5050/session/join", data=params)
if join.status_code == 200:
    print("join Success!")
    startParams = {
        "key": key, 
        "playername": "dohyun",
        "turn": 100,
        "dilation": 100
    }
    join = requests.post("http://20.196.214.79:5050/game/start", data=startParams)
    if join.status_code == 200:
        print("game start")
    else:
        print("game start Error")
else:
    print("join Error")


join Success!
game start


In [17]:
del GameMap

In [8]:
class GameMap:
    def __init__(self):
        self.game_map = []
        for _ in range(32):
            self.game_map.append([0]*32)
    
    def __str__(self):
        ret = '-'*60 + '\n'
        for row in self.game_map:
            ret += ' '.join(map(str, row))
            ret += '\n'
        ret = ret.replace('0', ' ')
        ret = ret.replace('1', 'T') # tank
        ret = ret.replace('2', '2')
        ret = ret.replace('3', 'O') # object
        ret = ret.replace('4', '4')
        ret = ret.replace('9', 'M') # my tank
        ret += '-'*60 + '\n'
        return ret
    
    def __repr__(self):
        return self.__str__()
    
    def set_map(self, x, y, objectType):
        x = (x-25200)//1000
        y = (y-147450)//1000
        
        try:
            assert 0 <= x <= 31 and 0 <= y <= 31
            self.game_map[y][x] = objectType
        except:
            print('exepted', x, y)

In [9]:
# game/status

statusParam = {"key": key, "playername": "dohyun"}
status = requests.get("http://20.196.214.79:5050/game/status", statusParam)
json_object = json.loads(status.content)
pprint.pprint(json_object)

{'key': 'd7d304350f3a5cbc07458bbd180b4c7bdc94ee7575e5cd34a252e908b5e8e49e',
 'requestCommand': 'status',
 'responses': {'data': {'id': 200,
                        'message': {'agent_info': {'agent': [{'ap': 10,
                                                              'hp': 100,
                                                              'location': [50200,
                                                                           157450],
                                                              'name': 'BP_K1A1_C_2147392515',
                                                              'uid': 111062},
                                                             {'ap': 10,
                                                              'hp': 100,
                                                              'location': [47200,
                                                                           157450],
                                                              'nam

In [10]:
# agent/rotate

agents = json_object['responses']["data"]["message"]["agent_info"]["agent"]
for idx in range(4):
    agentInfo = agents[idx]
    
    rotateParam = {"key": key, "uid": agentInfo["uid"], "direction": 45}
    while True:
        try:
            rotate = requests.post("http://20.196.214.79:5050/agent/rotate", data=rotateParam)
        except ConnectionRefusedError or ConnectionResetError:
            continue
        if rotate.status_code == 200:
            break
    print(agentInfo["name"], " -> 45")

BP_K1A1_C_2147392515  -> 45
BP_K1A1_C_2147392509  -> 45
BP_K1A1_C_2147392503  -> 45
BP_K1A1_C_2147392497  -> 45


In [14]:
# agent/attack
import time

agents = json_object['responses']["data"]["message"]["agent_info"]["agent"]
for idx in range(4):
    time.sleep(5)
    agentInfo = agents[idx]
    
    rotateParam = {"key": key, "uid": agentInfo["uid"]}
    while True:
        try:
            rotate = requests.post("http://20.196.214.79:5050/agent/attack", data=rotateParam)
        except ConnectionRefusedError or ConnectionResetError:
            continue
        if rotate.status_code == 200:
            break
    print(agentInfo["name"], " -> attack")

BP_K1A1_C_2147392515  -> attack
BP_K1A1_C_2147392509  -> attack
BP_K1A1_C_2147392503  -> attack
BP_K1A1_C_2147392497  -> attack


In [56]:
# agent/move

agents = json_object['responses']["data"]["message"]["agent_info"]["agent"]
for idx in range(4):
    agentInfo = agents[idx]

    direction = 0 # 전진
    if direction == 0:
        dirTxt = "Foword"
    elif direction == 1:
        dirTxt = "Left"
    elif direction == 2:
        dirTxt = "Back"
    elif direction == 3:
        dirTxt = "Right"
    
    moveParam = {"key": key, "uid": agentInfo["uid"], "direction": direction}
    while True:
        try:
            move = requests.post("http://20.196.214.79:5050/agent/move", data=moveParam)
        except ConnectionRefusedError or ConnectionResetError:
            continue
        if move.status_code == 200:
            break
    print(agentInfo["name"], " -> ", dirTxt)

BP_K1A1_C_2147392389  ->  Foword
BP_K1A1_C_2147392383  ->  Foword
BP_K1A1_C_2147392377  ->  Foword
BP_K1A1_C_2147392371  ->  Foword


In [17]:
agents = json_object['responses']["data"]["message"]["agent_info"]["agent"]
for idx in range(4):
    agentInfo = agents[idx]
    
    moveParam = {"key": key, "uid": agentInfo["uid"]}
    while True:
        try:
            move = requests.post("http://20.196.214.79:5050/agent/attack", data=moveParam)
        except ConnectionRefusedError or ConnectionResetError:
            continue
        if move.status_code == 200:
            break
    print(agentInfo["name"], " -> Attack")

In [8]:
# game/view 하나만 출력

agents = json_object['responses']["data"]["message"]["agent_info"]["agent"]
agentInfo = agents[0]
uid = agentInfo["uid"]
view = requests.get(f"http://20.196.214.79:5050/game/view", params={"key": key, "uid": uid})
info = json.loads(view.content)["responses"]["data"]["message"]["info"]
pprint.pprint(info)

[{'IsExistObject': False,
  'ObjectName': '',
  'ObjectType': 0,
  'location': [45200, 155450]},
 {'IsExistObject': False,
  'ObjectName': '',
  'ObjectType': 0,
  'location': [45200, 156450]},
 {'IsExistObject': True,
  'ObjectName': 'SM_CrashAreaBlock_2',
  'ObjectType': 3,
  'location': [45200, 157450]},
 {'IsExistObject': False,
  'ObjectName': '',
  'ObjectType': 0,
  'location': [45200, 158450]},
 {'IsExistObject': False,
  'ObjectName': '',
  'ObjectType': 0,
  'location': [46200, 153450]},
 {'IsExistObject': True,
  'ObjectName': 'Gop_ROKarmy_Wall_54M_69',
  'ObjectType': 3,
  'location': [46200, 154450]},
 {'IsExistObject': False,
  'ObjectName': '',
  'ObjectType': 0,
  'location': [46200, 155450]},
 {'IsExistObject': False,
  'ObjectName': '',
  'ObjectType': 0,
  'location': [46200, 156450]},
 {'IsExistObject': False,
  'ObjectName': '',
  'ObjectType': 0,
  'location': [46200, 157450]},
 {'IsExistObject': False,
  'ObjectName': '',
  'ObjectType': 0,
  'location': [46200, 

In [11]:
# game/view로 지도그리기

agents = json_object['responses']["data"]["message"]["agent_info"]["agent"]

game_map = GameMap()
objects = [] # view의 결과물들, 탱크 및 장애물들 리스트
for idx in range(4):
    agentInfo = agents[idx]
    uid = agentInfo["uid"]
    while True:
        try:
            view = requests.get(f"http://20.196.214.79:5050/game/view", params={"key": key, "uid": uid})
        except ConnectionRefusedError or ConnectionResetError:
            continue
        if view.status_code == 200:
            break
    
    info = json.loads(view.content)["responses"]["data"]["message"]["info"]
    print(len(info), len([x for x in info if x["IsExistObject"]]))
    
    for object in info:
        if object not in objects and object["IsExistObject"] == True:
            game_map.set_map(object['location'][0], object['location'][1], object['ObjectType'])
            objects.append(object)

for idx in range(4):
    agentInfo = agents[idx]
    game_map.set_map(agentInfo['location'][0], agentInfo['location'][1], 9)

print('total', len(objects))
print(game_map)

100 30
100 26
95 24
exepted -1 19
90 36
total 83
------------------------------------------------------------
                                                               
                                                               
                                                               
                                                               
                                                               
      O O O O                           O                      
        O O O                         O O       O O O O        
  O O O                               O O O     O O O O        
  O O O                               O             O O O O    
O O O O                           O O O             O O O O    
O         M     M                 O O O O   M     M O O O O    
  O O O O O O O                                                
  O O O O O O O                                                
                  O O                                     

In [12]:
objects

[{'location': [45200, 157450],
  'IsExistObject': True,
  'ObjectType': 3,
  'ObjectName': 'SM_CrashAreaBlock_2'},
 {'location': [46200, 154450],
  'IsExistObject': True,
  'ObjectType': 3,
  'ObjectName': 'Gop_ROKarmy_Wall_54M_69'},
 {'location': [47200, 157450],
  'IsExistObject': True,
  'ObjectType': 1,
  'ObjectName': 'BP_K1A1_C_2147392509'},
 {'location': [48200, 161450],
  'IsExistObject': True,
  'ObjectType': 3,
  'ObjectName': 'SM_Pile9_60'},
 {'location': [49200, 153450],
  'IsExistObject': True,
  'ObjectType': 3,
  'ObjectName': 'SM_Tower_19'},
 {'location': [49200, 154450],
  'IsExistObject': True,
  'ObjectType': 3,
  'ObjectName': 'Gop_ROKarmy_Wall_54M_70'},
 {'location': [50200, 153450],
  'IsExistObject': True,
  'ObjectType': 3,
  'ObjectName': 'SM_Barrels02_Merged_82'},
 {'location': [50200, 154450],
  'IsExistObject': True,
  'ObjectType': 3,
  'ObjectName': 'SM_Barrels02_Merged_82'},
 {'location': [51200, 153450],
  'IsExistObject': True,
  'ObjectType': 3,
  'Obj